In [1]:
import seaborn
from matplotlib import pyplot

In [2]:
import properties
import torch
import torch.autograd
import scipy.optimize

In [3]:
import tmol.io.pdb_parsing as pdb_parsing
import tmol.tests.data.pdb as test_pdbs

import tmol.minimize
import tmol.score
import tmol.system
import tmol.extern.py3dmol as py3dmol

from tmol.viewer import SystemViewer
from tmol.minimize import SimpleMinimizer
from tmol.system import FixedNamedAtomSystem
from tmol.score import ScoreGraph
from tmol.properties.array import Array

In [4]:
system = FixedNamedAtomSystem().load_pdb(test_pdbs.get("1ubq"))

In [5]:
import copy

In [6]:
score_graph = ScoreGraph(
    bond_graph = system.bond_graph,
    coords = torch.autograd.Variable(torch.Tensor(system.coords), requires_grad=True),
    r_m=2.1
)

In [7]:
minimizer = SimpleMinimizer(system=score_graph)

In [8]:
viewer = SystemViewer(system, score=copy.copy(score_graph))

In [9]:
from IPython.display import display, clear_output

def prop(_):
    system.coords = score_graph.coords.detach().numpy()
    viewer.update()
    
minimizer.minimize(options=dict(maxiter=100), callback=prop)

In [ ]:
seaborn.heatmap(score_graph.lj.detach().numpy())

In [ ]:
sg = tmol.score.ScoreGraph(
    bonds = s.model.bonds,
    coords = torch.autograd.Variable(torch.Tensor(s.model.coords), requires_grad=True)
)

optimizer = torch.optim.LBFGS((sg.coords,), max_iter=100, lr=1)

def step():
    print("step")
    optimizer.zero_grad()
    
    c = sg.coords
    sg.coords = None
    sg.coords = c
    
    total_score = sg.total_score
    total_score.backward()
    
    return total_score